# Power Outages
* **See the main project notebook for instructions to be sure you satisfy the rubric!**
* See Project 03 for information on the dataset.
* A few example prediction questions to pursue are listed below. However, don't limit yourself to them!
    * Predict the severity (number of customers, duration, or demand loss) of a major power outage.
    * Predict the cause of a major power outage.
    * Predict the number and/or severity of major power outages in the year 2020.
    * Predict the electricity consumption of an area.

Be careful to justify what information you would know at the "time of prediction" and train your model using only those features.

# Summary of Findings


### Introduction
I think we should try to predict outage duration

### Baseline Model
TODO

### Final Model
TODO

### Fairness Evaluation
TODO

# Code

In [126]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin



%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # Higher resolution figures

### Baseline Model

variables to predict with: POSTAL.CODE, CAUSE.CATEGORY, TOTAL.CUSTOMERS, TOTAL.PRICE, TOTAL.SALES, PC.REALGSP.STATE, TOTAL.REALGSP, AREAPCT_URBAN

In [2]:
#importing data and putting it into the correct format
data = pd.read_excel('outage.xlsx')

data = data.drop(index=[0,1,2,3,5], axis=0)
new_header = data.iloc[0]
data = data[1:]
data.columns = new_header
data = data.drop(labels='variables', axis=1)
data = data.reset_index()

In [51]:
subset = data[['OUTAGE.DURATION','POSTAL.CODE', 'CAUSE.CATEGORY', 'TOTAL.CUSTOMERS', 'TOTAL.PRICE', 'TOTAL.SALES', 'PC.REALGSP.STATE', 'TOTAL.REALGSP', 'AREAPCT_URBAN', 'ANOMALY.LEVEL']]

In [52]:
subset.isnull().sum()

4
OUTAGE.DURATION     58
POSTAL.CODE          0
CAUSE.CATEGORY       0
TOTAL.CUSTOMERS      0
TOTAL.PRICE         22
TOTAL.SALES         22
PC.REALGSP.STATE     0
TOTAL.REALGSP        0
AREAPCT_URBAN        0
ANOMALY.LEVEL        9
dtype: int64

In [49]:
data

4,index,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,...,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
0,6,1,2011,7,Minnesota,MN,MRO,East North Central,-0.3,normal,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5927,8.40733,5.47874
1,7,2,2014,5,Minnesota,MN,MRO,East North Central,-0.1,normal,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5927,8.40733,5.47874
2,8,3,2010,10,Minnesota,MN,MRO,East North Central,-1.5,cold,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5927,8.40733,5.47874
3,9,4,2012,6,Minnesota,MN,MRO,East North Central,-0.1,normal,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5927,8.40733,5.47874
4,10,5,2015,7,Minnesota,MN,MRO,East North Central,1.2,warm,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5927,8.40733,5.47874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1529,1535,1530,2011,12,North Dakota,ND,MRO,West North Central,-0.9,cold,...,59.9,19.9,2192.2,1868.2,3.9,0.27,0.1,97.5996,2.40177,2.40177
1530,1536,1531,2006,NaN,North Dakota,ND,MRO,West North Central,NaN,NaN,...,59.9,19.9,2192.2,1868.2,3.9,0.27,0.1,97.5996,2.40177,2.40177
1531,1537,1532,2009,8,South Dakota,SD,RFC,West North Central,0.5,warm,...,56.65,26.73,2038.3,1905.4,4.7,0.3,0.15,98.3077,1.69226,1.69226
1532,1538,1533,2009,8,South Dakota,SD,MRO,West North Central,0.5,warm,...,56.65,26.73,2038.3,1905.4,4.7,0.3,0.15,98.3077,1.69226,1.69226


In [53]:
#handling null values
#subset.isnull().sum()

columns_with_nulls = ['OUTAGE.DURATION', 'TOTAL.PRICE', 'TOTAL.SALES']

for col in columns_with_nulls:
    num_null = subset[col].isnull().sum()
    fills = subset[col].dropna().sample(num_null, replace = True)
    fills.index = subset.loc[subset[col].isnull()].index
    subset = subset.fillna({col:fills.to_dict()})



In [70]:
ct1 = ColumnTransformer(
    [
        ('ohe', OneHotEncoder(handle_unknown = 'ignore'), ['POSTAL.CODE', 'CAUSE.CATEGORY']),
    ])
pl1 = Pipeline(steps = [
            ('columntrans', ct),
            ('reg', LinearRegression())
            ])

In [71]:
x = subset.drop('OUTAGE.DURATION', axis=1)
y = subset['OUTAGE.DURATION']
x_train, x_test, y_train, y_test = train_test_split(x, y)

pl1.fit(x_train, y_train)



Pipeline(memory=None,
     steps=[('columntrans', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('ohe', OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values=None, sparse=True), ['POSTAL.CODE', 'CAUSE.CATEGORY'])])), ('reg', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False))])

In [72]:
pred = pl1.predict(x_test)

In [64]:
pl1.named_steps['reg'].coef_

array([16332.96102316, -1350.82872342, -1142.75454037,  4552.99469351,
       -1122.84550098, -1428.1521702 ,  -978.67150553,   241.2927199 ,
        -600.22436588,   882.42167078, -2265.14505402, -2564.63792701,
        3787.57935833,  -318.25365889, -2360.04448271,   457.43326204,
         333.51893061,   841.85935161,   922.82229637, -1214.45316877,
          69.9771147 , -1731.84579757,  1887.20830195,  -331.69890715,
         199.75712587,  -557.52797745,  -752.4853572 , -2046.14951552,
        -646.059583  ,  -892.19596144,  -836.132017  ,   966.60175109,
       -2078.09068177,   -26.77474287,  3054.85053406,  -105.79250343,
        -974.87805066,  -523.62187249,   324.04488888, -1435.15572858,
       -1025.42694824, -1850.20172377,  -335.7474429 ,  -636.03891003,
       -2666.31648567,  -534.08402307,    93.08239014,  -764.1148029 ,
        3857.12850732, -1676.23510318,  -143.01934597,  5551.23500365,
       -1957.53154678, -1442.6303073 , -1053.22264374,  1445.595177  ,
      

In [48]:
subset

4,OUTAGE.DURATION,POSTAL.CODE,CAUSE.CATEGORY,TOTAL.CUSTOMERS,TOTAL.PRICE,TOTAL.SALES,PC.REALGSP.STATE,TOTAL.REALGSP,AREAPCT_URBAN
0,3060.0,MN,severe weather,2595696,9.28,6562520.0,51268,274182,2.14
1,1.0,MN,intentional attack,2640737,9.28,5284231.0,53499,291955,2.14
2,3000.0,MN,severe weather,2586905,8.15,5222116.0,50447,267895,2.14
3,2550.0,MN,severe weather,2606813,9.19,5787064.0,51598,277627,2.14
4,1740.0,MN,severe weather,2673531,10.43,5970339.0,54431,292023,2.14
...,...,...,...,...,...,...,...,...,...
1529,720.0,ND,public appeal,394394,7.56,1313678.0,57012,39067,0.27
1530,2490.0,ND,fuel supply emergency,366037,11.08,7583977.0,42913,27868,0.27
1531,59.0,SD,islanding,436229,7.67,924051.0,45230,36504,0.3
1532,181.0,SD,islanding,436229,7.67,924051.0,45230,36504,0.3


In [50]:
y_test

884        0.0
91      4590.0
829      989.0
1476      75.0
488     2520.0
         ...  
795     7020.0
1042    4320.0
450      255.0
1146    4890.0
1          1.0
Name: OUTAGE.DURATION, Length: 384, dtype: float64

In [73]:

r2_score(y_test, pred)

0.10904147829891697

# Trying to classify cause of hurricane

In [152]:
subset = data[['CAUSE.CATEGORY', 'OUTAGE.DURATION','POSTAL.CODE', 'TOTAL.CUSTOMERS', 'MONTH', 'PC.REALGSP.STATE', 'UTIL.CONTRI', 'AREAPCT_URBAN', 'ANOMALY.LEVEL', 'CLIMATE.REGION','TOTAL.SALES']]

In [20]:
len(data)

1534

In [157]:
subset.isnull().sum()

4
CAUSE.CATEGORY      0
OUTAGE.DURATION     0
POSTAL.CODE         0
TOTAL.CUSTOMERS     0
MONTH               0
PC.REALGSP.STATE    0
UTIL.CONTRI         0
AREAPCT_URBAN       0
ANOMALY.LEVEL       0
CLIMATE.REGION      0
TOTAL.SALES         0
dtype: int64

In [154]:
#filling null values of numeric variables with probabilistic imputation
columns_with_nulls = ['OUTAGE.DURATION', 'ANOMALY.LEVEL', 'TOTAL.SALES']

for col in columns_with_nulls:
    num_null = subset[col].isnull().sum()
    fills = subset[col].dropna().sample(num_null, replace = True)
    fills.index = subset.loc[subset[col].isnull()].index
    subset = subset.fillna({col:fills.to_dict()})



In [156]:
#dropping rows with null values in categorical variables
subset = subset.dropna()

In [22]:
subset

4,CAUSE.CATEGORY,OUTAGE.DURATION,POSTAL.CODE,TOTAL.CUSTOMERS,MONTH,PC.REALGSP.STATE,UTIL.CONTRI,AREAPCT_URBAN,ANOMALY.LEVEL,CLIMATE.REGION,TOTAL.SALES
0,severe weather,3060.0,MN,2595696,7,51268,1.75139,2.14,-0.3,East North Central,6562520.0
1,intentional attack,1.0,MN,2640737,5,53499,1.79,2.14,-0.1,East North Central,5284231.0
2,severe weather,3000.0,MN,2586905,10,50447,1.70627,2.14,-1.5,East North Central,5222116.0
3,severe weather,2550.0,MN,2606813,6,51598,1.93209,2.14,-0.1,East North Central,5787064.0
4,severe weather,1740.0,MN,2673531,7,54431,1.6687,2.14,1.2,East North Central,5970339.0
...,...,...,...,...,...,...,...,...,...,...,...
1527,intentional attack,3600.0,ID,849763,3,36256,1.57607,0.6,1.6,Northwest,1657605.0
1528,system operability disruption,220.0,ID,849763,7,36256,1.57607,0.6,-0.3,Northwest,447034.0
1529,public appeal,720.0,ND,394394,12,57012,2.39076,0.27,-0.9,West North Central,1313678.0
1531,islanding,59.0,SD,436229,8,45230,1.66009,0.3,0.5,West North Central,924051.0


In [158]:
ct = ColumnTransformer(
    [
        ('ohe', OneHotEncoder(handle_unknown = 'ignore'), ['POSTAL.CODE', 'CLIMATE.REGION']),
    ])
pl = Pipeline(steps = [
            ('columntrans', ct),
            ('knn',  KNeighborsClassifier(n_neighbors = 5))
            ])

In [164]:
#finding average average accuracy

accuracies = []
for i in range(100):
    y = subset['CAUSE.CATEGORY']
    x = subset.drop('CAUSE.CATEGORY', axis=1)
    x_train, x_test, y_train, y_test = train_test_split(x, y)
    pl.fit(x_train, y_train)
    accuracies.append(accuracy_score(y_test, pl.predict(x_test)))

In [160]:
pd.Series(accuracies).mean()

0.544078947368421

Is this good? Let's compare it to the accuracy from randomly guessing

In [165]:
y = subset['CAUSE.CATEGORY']
x = subset.drop('CAUSE.CATEGORY', axis=1)
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [173]:
0.25*len(subset)


380.0

In [179]:
accuracies = []
for i in range(100):
    #The 0.25 is because train_test_split uses 0.25 of the data for testing as default
    guesses = subset['CAUSE.CATEGORY'].sample(n = int(0.25*len(subset)), replace = True)
    
    y = subset['CAUSE.CATEGORY']
    x = subset.drop('CAUSE.CATEGORY', axis=1)
    x_train, x_test, y_train, y_test = train_test_split(x, y)
    accuracies.append(accuracy_score(y_test, guesses))

In [180]:
pd.Series(accuracies).mean()

0.33463157894736834

looks like its better than randomly guessing!

### Final Model

First, let's determine the optimal number of neighbors to classify on. In the baseline model we only used 5.

In [103]:
y = subset['CAUSE.CATEGORY']
x = subset.drop('CAUSE.CATEGORY', axis=1)
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [122]:
parameters = {
    'knn__n_neighbors': [i for i in range(1,51)]
}

In [123]:
ct = ColumnTransformer(
    [
        ('ohe', OneHotEncoder(handle_unknown = 'ignore'), ['POSTAL.CODE', 'CLIMATE.REGION']),
    ])
pl = Pipeline(steps = [
            ('columntrans', ct),
            ('knn',  KNeighborsClassifier())
            ])

In [124]:
clf = GridSearchCV(pl, parameters, cv = 5)

In [125]:
clf.fit(x_train, y_train)

C:\Users\ianma\Downloads\anaconda\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('columntrans', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('ohe', OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='ignore',
       n_values=Non...ki',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'knn__n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [130]:
best_k = clf.best_params_['knn__n_neighbors']

38

In [ ]:
#scale anomoly level by climate region
#month is nominal so we should change that to string and do OHE
#maybe remove outliers

In [131]:


class StdScalerByGroup(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass

    def fit(self, X, y=None):
        # X may not be a pandas dataframe (e.g. a np.array)
        df = pd.DataFrame(X)
        
        # A dictionary of means/standard-deviations for each column, for each group.
        
        #use X.columns[0] since groups are always in first column
        agg = df.groupby(X.columns[0]).agg(['mean', 'std'])

        self.grps_ = agg.to_dict()

        return self

    def transform(self, X, y=None):

        try:
            getattr(self, "grps_")
        except AttributeError:
            raise RuntimeError("You must fit the transformer before tranforming the data!")
        

        # Define a helper function here?
        def z_helper(series):
            #we need column name and group. How to access?
            #to get the group name use series.name
            group_name = series.name
            series = series.set_index(series.columns[0])
            def standardize(col):
                mean = self.grps_[(col.name, 'mean')][group_name]
                std = self.grps_[(col.name, 'std')][group_name]
                standardized = (col - mean) / std
                return standardized

            return series.apply(standardize)
            
        
        # X may not be a dataframe (e.g. np.array)

        df = pd.DataFrame(X)
        
        return df.groupby(df.columns[0]).apply(z_helper)
        

In [151]:
def to_string(x):
        return pd.DataFrame(x).astype(str)
num_to_str = FunctionTransformer(func = to_string, validate=False)

month_trans = Pipeline(steps = [
            ('to_string', num_to_str),
            #added categories='auto' to silence warning message
            #handle_unknown incase there are unseen values in the test set
            ('ohe', OneHotEncoder(handle_unknown = 'ignore', categories='auto'))
            ])

ct = ColumnTransformer(
    [
        ('ohe', OneHotEncoder(handle_unknown = 'ignore', categories='auto'), ['POSTAL.CODE', 'CLIMATE.REGION']),
        ('std', StdScalerByGroup(), ['CLIMATE.REGION','ANOMALY.LEVEL']),
        ('month_trans', month_trans, ['MONTH'])
        
    ])
pl = Pipeline(steps = [
            ('columntrans', ct),
            ('knn',  KNeighborsClassifier(n_neighbors = best_k))
            ])

In [150]:
#finding average average accuracy

accuracies = []
for i in range(100):
    y = subset['CAUSE.CATEGORY']
    x = subset.drop('CAUSE.CATEGORY', axis=1)
    x_train, x_test, y_train, y_test = train_test_split(x, y)
    pl.fit(x_train, y_train)
    accuracies.append(accuracy_score(y_test, pl.predict(x_test)))

In [148]:
pd.Series(accuracies).mean()

0.5944473684210523

### Fairness Evaluation

In [ ]:
# TODO